In [1]:
from mtasklite import Pool, delayed_init, is_exception, set_process_start_method, ExceptionBehaviour, ArgumentPassing
# Use of tqdm is optional!
from tqdm.auto import tqdm 

In [2]:
# It's really important to do so on Mac and Windows
set_process_start_method()

In [10]:
@delayed_init
class SimpleArithmeticClassWorker:
    def __init__(self, proc_id):
        print(f'Init. process {proc_id}\n')
        self.proc_id = proc_id

    def __call__(self, input_arg):
        ret_val = input_arg
        for t in range(100_000):
            ret_val = (ret_val * ret_val) % 337
        return ret_val

In [15]:
N_TASKS = 1000      
N_JOBS = 4
    
input_arr = [k * 10 for k in range(N_TASKS)]
function_or_worker_arr = [SimpleArithmeticClassWorker(proc_id) for proc_id in range(N_JOBS)]

In [17]:
tot_res = 0    

with Pool(function_or_worker_arr,
         argument_passing=ArgumentPassing.AS_SINGLE_ARG,
         exception_behavior=ExceptionBehaviour.DEFERRED,
         is_unordered=False,
         join_timeout=1) as proc_pool:

    for result in tqdm(proc_pool(input_arr)):
        # With deferred exceptions, they are simply returned as a part of the result object
        if is_exception(result):
            print('Error:', type(result), str(result))
        else:
            tot_res += result

print('Total:', tot_res)

  0%|          | 0/1000 [00:00<?, ?it/s]

Init. process 0

Total: 112152
